In [188]:
import requests
conn = {'username':'guest',
        'password':'guest',
        'api':'1',
         # vamps:             https://vamps2.mbl.edu
         # vampsdev (private) http://vampsdev.jbpc-np.mbl.edu:8124 
         # localhost:         http://localhost:3000 
        'hosturl':'http://localhost:3000'
       }
s = requests.Session()
r = s.post(conn['hosturl']+'/api/logmein', data=conn)
r = s.post(conn['hosturl']+'/api/validate_login', data=conn)
r.text

u'Login Success'

In [189]:
"""
Panel-1 
Define config variable; Select project; Select Image;
Gets datasets ids
Author: AAV
vamps2.mbl.edu
"""
import os,sys
import json
from bs4 import BeautifulSoup   # parser for html

#ready_images = ["dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "phyloseq","adiversity","dendrogram","fheatmap"]
config = {
    "api":"1",
    "source":"VAMPS-API",
    "update_data":1,
    "normalization":"none",                  # none, maximum, frequency
    "selected_distance":"morisita-horn",     # morisita-horn, jaccard, kulczynski, canberra, bray-curtis
    "tax_depth":"genus",                     # domain, phylum, klass, order, family, genus, species, strain
    "domains":["Archaea","Bacteria","Eukarya","Organelle","Unknown"],   #["Archaea","Bacteria","Eukarya","Organelle","Unknown"],
    "include_nas":"yes",                  # yes or no
    # if the % taxon for all of the datasets are outside this range the taxon will not show.
    # Applies to pies and bars, heatmap, tax-table:
    "min_range":0,                        # integer 0-99  -> percent range of taxonomy to show
    "max_range":100,                      # integer 1-100 -> percent range of taxonomy to show
    
    # CHOOSE EITHER 'project' OR 'ds_order'.
    # ds-order takes precedence and allows you to get data from partial projects or multiple projects.
    # find dataset_ids from the GUI or use the 'get_dids_from_project' function (see next panel)
    # Must be a valid project - with correct permissions for the above user.
    #'project':'AFP_MVCO_Bv6',    # localhost tests: KCK_LSM_TBS(private) or ICM_SPO_Ev9 or ICM_LCY_Bv6   
                                 # vamps2: AB_SAND_Bv6 AFP_MVCO_Bv6 DCO_BRG_Bv6v4
    #  ds_order format: [49, 50, 51, 52],  (no quotes)
    'ds_order':[49, 50, 51], # IF PRESENT and not empty - will take precedence over 'project'

    # Currently avalable: 
    # "dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "adiversity", "dendrogram", "fheatmap", "taxon_color_legend"
    # In construction    phyloseq (Rscript)
    'image':'taxon_color_legend',
}
# minimal config:
config = {
    "api":"1",
    "source":"VAMPS-API",
    "update_data":1,
    'ds_order':[49, 50, 51],  ## OR 'project':'AFP_MVCO_Bv6',
    'image':'piecharts',    
}
print('Ready')

Ready


In [190]:
"""
API function listing
"""
# ############################################
# VAMPS-API: Get Dataset IDs:
# data = {"project":"ICM_LCY_Bv6"}
# r = s.post(conn['hosturl']+'/api/get_dids_from_project', timeout=15, data=data)  
# result = json.loads(r.text)
# result
# ############################################
# VAMPS-API: Get Project Metadata:
# Will return metadata in JSON format. 
# If you want a tabular csv file use 'image':'metadata_csv' with the create_image function
# data = {"project":"ICM_LCY_Bv6"} 
# r = s.post(conn['hosturl']+'/api/get_metadata_from_project', timeout=15, data=data)  
# result = json.loads(r.text)
# result
# ############################################
# # VAMPS-API: Get Project Information:
# data = {"project":"KCK_LSM_TBS"}
# r = s.post(conn['hosturl']+'/api/get_project_information', timeout=15, data=data)  
# result = json.loads(r.text)
# result
# ############################################
#VAMPS-API: Find Projects that user has access to.
# data = {
#    'search_string':'',  # If not empty will return projects with string in project name (case insensitive)
#                         # title and description
#     #'include_info':''   # if present, data will include project information
# }
# r = s.post(conn['hosturl']+'/api/find_user_projects', timeout=15, data=data) 
# result = json.loads(r.text)
# result
############################################
#VAMPS-API: Get Projects in Geographic Region
#data: JSON Decimal Degrees; 
# data = {'nw_lat':'90','nw_lon':'-165','se_lat':'','se_lon':''}
# r = s.post(conn['hosturl']+'/api/find_projects_in_geo_area', timeout=15, data=data)  
# result = json.loads(r.text)
# result
############################################
#VAMPS-API: FIND PROJECTS BY METADATA STRING
#data: JSON substring to search all project metadata 
# data = {'substring':'aux'}
# r = s.post(conn['hosturl']+'/api/find_projects_by_metadata_str', timeout=15, data=data)  
# result = json.loads(r.text)
# result


'\nAPI function listing\n'

In [191]:
"""
Panel-2
Creates remote configuration and gets ts (file_prefix) 
"""
# Get timestamp (filename prefix) and setup environment:
r = s.post(conn['hosturl']+'/visuals/view_selection', timeout=15, data=config)
soup = BeautifulSoup(r.text, "lxml")  # html5lib  lxml html.parser

# I placed this on view_selection page:  <div id='ts_for_bs' style='visibility:hidden'><%= pi.ts %></div>
found_data = str(soup.find(id="found_data").string)
if found_data == 'None':
    print("\x1b[31m\"No Data\"\x1b[0m"+" -- verify correct 'config' structure and permissions for data you are trying to access" )
    sys.exit()
ts = str(soup.find(id="ts_for_bs").string)
ts

'guest_1503493547295'

In [192]:
"""
Panel-3
Writes local matrx_file which is integral to vamps images:
Writes image file
"""
import json
biom_matrix_file = ts+'_count_matrix.biom'
url = conn['hosturl']+"/"+biom_matrix_file
response = requests.get(url, stream=True)
response.raise_for_status()
out_file = biom_matrix_file
with open(out_file, "wb") as handle:
    for block in response.iter_content(1024):
        handle.write(block)

r = s.post(conn['hosturl']+'/api/create_image', timeout=300, data=config)

try:
    result = json.loads(r.text)
except:
    print(r.text)
    sys.exit()
local_filename = result['filename']
return_result = result['html']
#print(local_filename)
remote_file_name = conn['hosturl']+"/"+local_filename

r = requests.get(remote_file_name, timeout=300, stream=True)
with open(local_filename, 'wb') as f:
    f.write(r.content)
'Done writing local file: '+local_filename

u'Done writing local file: guest_1503493547295-piecharts-api.svg'

In [193]:
"""
Show image
"""
from IPython.core.display import display, HTML
out = ''

if config['image'] == 'metadata_csv':
    print(return_result)
else:
    #display(HTML("<style>.container { width:100% !important; }</style>"+return_result))
    out = HTML("<style>.container { width:100% !important; }</style>"+return_result)

out